In [1]:
%%javascript
Jupyter.utils.load_extensions('tdb_ext/main')

<IPython.core.display.Javascript object>

In [2]:
# coding: utf-8
import tensorflow as tf
import prettytensor as pt
import numpy as np
import cmtf.data.data_mnist as data_mnist
import matplotlib.pyplot as plt
import tdb
from tdb.examples import viz


# 模型
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int64, [None])

x_reshape = tf.reshape(x, [-1, 28, 28, 1])
seq = pt.wrap(x_reshape).sequential()
seq.conv2d([7, 7], 25, activation_fn=tf.nn.relu)
seq.max_pool(2, 2)
seq.conv2d([7, 7], 32, activation_fn=tf.nn.relu)
seq.max_pool(2, 2)
seq.flatten()
seq.fully_connected(64, activation_fn=tf.nn.relu)
seq.fully_connected(10, activation_fn=None)
output = seq.as_layer()

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(output, y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output,1), y), "float"))
train_op = tf.train.MomentumOptimizer(0.01, 0.9, use_nesterov=True).minimize(loss)

# 所有变量
for v in tf.all_variables():
	print v.name

# GPU使用率
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8    #固定比例
config.gpu_options.allow_growth = True

# session
sess = tf.InteractiveSession(config=config)
tf.initialize_all_variables().run()

# 画图
def viz_square(data, normalize=True, cmap=plt.cm.gray, padsize=1, padval=0):
    # normalize to 0-1 range
    if normalize:
        data -= data.min()
        data /= data.max()
    n = int(np.ceil(np.sqrt(data.shape[0]))) # force square 
    padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    plt.matshow(data,cmap=cmap)

# 最大激活权重
def max_activate_weight(weights):
    image = np.copy(weights)
    for i in range(image.shape[0]):
        base = np.sqrt(np.square(image[i]).sum())
        image[i] = np.absolute(image[i]) / base
    return image

def viz_conv_weights(ctx, weight):
    weight_tr = weight.transpose(3,0,1,2)[:,:,:,0]
    images = max_activate_weight(weight_tr)
#     viz_square(images, normalize=False)
    plt.matshow(images[0],cmap=plt.cm.gray)

    
g=tf.get_default_graph()
conv1_weights = [v for v in tf.all_variables() if v.name == "conv2d/weights:0"][0]
p1=tdb.plot_op(viz_conv_weights,inputs=[g.as_graph_element(conv1_weights)])

ploss=tdb.plot_op(viz.watch_loss,inputs=[loss])

print conv1_weights.get_shape()

# 数据
mnist = data_mnist.read_data_sets(one_hot=False)

# train
for i in range(100):
    batch_xs, batch_ys = mnist.train.next_batch(32)
    feed_dict={x: batch_xs, y: batch_ys}
    train_op.run(feed_dict=feed_dict)
    if (i+1)%10 == 0:
        status, result=tdb.debug([p1, ploss], feed_dict=feed_dict, breakpoints=None, 
                                 break_immediately=False, session=sess)

# test
accuracy_value = accuracy.eval(feed_dict={x:mnist.test.images[:100], y:mnist.test.labels[:100]})
print 'Accuracy: %g' % accuracy_value

conv2d/weights:0
conv2d/bias:0
conv2d_1/weights:0
conv2d_1/bias:0
fully_connected/weights:0
fully_connected/bias:0
fully_connected_1/weights:0
fully_connected_1/bias:0
conv2d/weights/Momentum:0
conv2d/bias/Momentum:0
conv2d_1/weights/Momentum:0
conv2d_1/bias/Momentum:0
fully_connected/weights/Momentum:0
fully_connected/bias/Momentum:0
fully_connected_1/weights/Momentum:0
fully_connected_1/bias/Momentum:0
(7, 7, 1, 25)
Extracting /Users/lichen/data/tf/mnist/train-images-idx3-ubyte.gz
Extracting /Users/lichen/data/tf/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/lichen/data/tf/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/lichen/data/tf/mnist/t10k-labels-idx1-ubyte.gz
Accuracy: 0.93
